# 测试

In [ ]:
import tvm

In [ ]:
import numpy as np
from IPython.display import display_svg
from tvm import te, build, lower
from tvm_book.testing.relay.viz import graphviz_relay
from tvm.contrib import tedd

In [ ]:
A = te.placeholder((1,), name="A")
B = te.placeholder((1,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")
sch = te.create_schedule(C.op)
ir_mod = lower(sch, [A, B, C], name="test_add")
rt_mod = build(ir_mod, target="llvm")
func = te.create_prim_func([A, B, C])
print(func.script())

In [ ]:
graph = tedd.viz_dataflow_graph(sch, show_svg=True)
display_svg(graph)

In [ ]:
sch = sch.normalize()
tree = tedd.viz_schedule_tree(sch, show_svg=True)

# tedd.viz_schedule_tree(s, dot_file_path="/tmp/scheduletree.dot")
display_svg(tree)

In [ ]:
from graphviz import Source
# tedd.viz_itervar_relationship_graph(s, dot_file_path="/tmp/itervar.dot")
dot_string = tedd.viz_itervar_relationship_graph(sch, output_dot_string=True)
src = Source(dot_string)
display_svg(src)

## `te.scan`

In [ ]:
m = te.var("m")
n = te.var("n")
X = te.placeholder((m, n), name="X")
s_state = te.placeholder((m, n))
s_init = te.compute((1, n), lambda _, i: X[0, i])
s_update = te.compute((m, n), lambda t, i: s_state[t-1, i] + X[t, i])
res = te.scan(s_init, s_update, s_state, X)
sch = te.create_schedule(res.op)
sch = sch.normalize() # 类型和 shape 推断

In [ ]:
from graphviz import Source
# tedd.viz_itervar_relationship_graph(s, dot_file_path="/tmp/itervar.dot")
dot_string = tedd.viz_itervar_relationship_graph(sch, output_dot_string=True)
src = Source(dot_string)
display_svg(src)

In [ ]:
tree = tedd.viz_schedule_tree(sch, show_svg=True)

# tedd.viz_schedule_tree(s, dot_file_path="/tmp/scheduletree.dot")
display_svg(tree)

In [ ]:
graph = tedd.viz_dataflow_graph(sch, show_svg=True)
display_svg(graph)

## 向量加法

In [ ]:
from tvm_book.contrib.d2ltvm import get_abc

In [ ]:
a, b, c = get_abc(shape=(1,))

In [ ]:
def vector_add(a, b, c):
    n = len(a)
    for i in range(n):
        c[i] = a[i] + b[i]

In [ ]:
vector_add(a, b, c)

In [ ]:
from tvm.runtime import load_module

In [ ]:
mod_fname = "add.tar"
rt_mod.export_library(mod_fname) # 保存
loaded_mod = load_module(mod_fname) # 加载

## 神经网络

In [1]:
import tensorflow as tf

2023-06-17 14:48:41.952203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 14:48:42.012931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 14:48:42.019192: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 14:48:42.952345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
tf.test.is_gpu_available()